In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [3]:
# Load the ANN trained model, scaler pickle file and one hot encoding model as well
model = load_model('model.h5')

## Load the encoder and scaler
with open('ohe_geo.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open("label_encoder_gender.pkl", 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [11]:
# We can pass some random input data.
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}
geo_encoded = ohe_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(["Geography"]))

C:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [13]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
# Encode categorical variables
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [16]:
## Concatenation onehotencoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [17]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
# Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([[0.08993216]], dtype=float32)

In [20]:
prediction_proba=prediction[0][0]
prediction_proba

0.08993216

In [21]:
if prediction_proba > 0.5:
    print("Customer likely to churn")
else:
    print("Customer not likely to churn")

Customer not likely to churn
